In [64]:
# import dependencies 
import pandas as pd
import numpy as np
import congi

## Read in csv and covert to json to dataframe

In [48]:
# Read in csv 
covid_data_df = pd.read_csv(r"date,county,state,fips,cases,deaths.csv")

# Convert csv to_json and name covid_data_df
covid_data_df.to_json(r"covid_data.json")
covid_data_df

,date,county,state,fips,cases,deaths
0,1/21/2020,Snohomish,Washington,53061.0,1,0.0
1,1/22/2020,Snohomish,Washington,53061.0,1,0.0
2,1/23/2020,Snohomish,Washington,53061.0,1,0.0
3,1/24/2020,Cook,Illinois,17031.0,1,0.0
4,1/24/2020,Snohomish,Washington,53061.0,1,0.0
...,...,...,...,...,...,...
785515,12/1/2020,Ontonagon,Michigan,26131.0,280,14.0
785516,12/1/2020,Osceola,Michigan,26133.0,559,10.0
785517,12/1/2020,Oscoda,Michigan,26135.0,190,8.0
785518,12/1/2020,Otsego,Michigan,26137.0,874,18.0


## Clean and organinze data

In [49]:
# Create dataFrame for only California 
covid_cali_df = covid_data_df.loc[covid_data_df["state"] == "California"]
covid_cali_df.head()

,date,county,state,fips,cases,deaths
5,1/25/2020,Orange,California,6059.0,1,0.0
9,1/26/2020,Los Angeles,California,6037.0,1,0.0
10,1/26/2020,Orange,California,6059.0,1,0.0
14,1/27/2020,Los Angeles,California,6037.0,1,0.0
15,1/27/2020,Orange,California,6059.0,1,0.0


In [50]:
# Get county names 
covid_cali_df.county.unique()

array(['Orange', 'Los Angeles', 'Santa Clara', 'San Francisco',
       'San Diego', 'Humboldt', 'Sacramento', 'Solano', 'Marin', 'Napa',
       'Sonoma', 'Alameda', 'Placer', 'San Mateo', 'Contra Costa', 'Yolo',
       'Fresno', 'Madera', 'Riverside', 'Santa Cruz', 'Shasta',
       'San Joaquin', 'Ventura', 'Stanislaus', 'Tulare', 'San Benito',
       'San Luis Obispo', 'San Bernardino', 'Santa Barbara', 'Nevada',
       'Kern', 'Monterey', 'Mendocino', 'Amador', 'Imperial', 'Butte',
       'El Dorado', 'Siskiyou', 'Yuba', 'Unknown', 'Calaveras', 'Merced',
       'Mono', 'Inyo', 'Sutter', 'Colusa', 'Kings', 'Glenn', 'Tuolumne',
       'Alpine', 'Plumas', 'Del Norte', 'Tehama', 'Lake', 'Mariposa',
       'Trinity', 'Sierra', 'Lassen', 'Modoc'], dtype=object)

In [51]:
# Clean data -  remove 'unknown'
clean_covid_df = covid_cali_df[covid_cali_df.county != 'Unknown']
clean_covid_df.head()

,date,county,state,fips,cases,deaths
5,1/25/2020,Orange,California,6059.0,1,0.0
9,1/26/2020,Los Angeles,California,6037.0,1,0.0
10,1/26/2020,Orange,California,6059.0,1,0.0
14,1/27/2020,Los Angeles,California,6037.0,1,0.0
15,1/27/2020,Orange,California,6059.0,1,0.0


In [52]:
# Confirm that only 58 counties in California
clean_covid_df.county.nunique()

58

## Create dataframes for socal and nocal counties

In [53]:
# Create socalArray for dataFrame with socal counties only
socalArray = ["Inyo", "Kern", "San Luis Obispo", "Santa Barbara", "Ventura", "Los Angeles", "San Bernardino", "Orange", "Riverside", "San Diego", "Imperial"]

# Creating only socal_data_df by selecting socal counties with isin
socal_data_df = clean_covid_df.loc[clean_covid_df["county"].isin(socalArray)]
socal_data_df.head()

,date,county,state,fips,cases,deaths
5,1/25/2020,Orange,California,6059.0,1,0.0
9,1/26/2020,Los Angeles,California,6037.0,1,0.0
10,1/26/2020,Orange,California,6059.0,1,0.0
14,1/27/2020,Los Angeles,California,6037.0,1,0.0
15,1/27/2020,Orange,California,6059.0,1,0.0


In [54]:
# Confirm that only the 11 socal counties are in dataFrame
socal_data_df.nunique()

date       312
county      11
state        1
fips        11
cases     2332
deaths     905
dtype: int64

In [55]:
# Using socal_data_df, merge with clean_covid_cali_df and create duplicates for socal counties
duplicates = pd.merge(clean_covid_df, socal_data_df, how= 'inner', left_on=['date', 'county'], right_on=['date', 'county'], left_index=True)

# Use .drop duplicates on index to create nocal_data_df
norcal_data_df = clean_covid_df.drop(duplicates.index)
norcal_data_df.head()

,date,county,state,fips,cases,deaths
36,1/31/2020,Santa Clara,California,6085.0,1,0.0
42,2/1/2020,Santa Clara,California,6085.0,1,0.0
49,2/2/2020,San Francisco,California,6075.0,2,0.0
50,2/2/2020,Santa Clara,California,6085.0,2,0.0
57,2/3/2020,San Francisco,California,6075.0,2,0.0


In [56]:
# Confirm that only 47 counties for nocal
norcal_data_df.nunique()

date       306
county      47
state        1
fips        47
cases     4297
deaths     465
dtype: int64

## Load to pgadmin

In [70]:
#clean_covid_df.dtypes

In [ ]:
# Make connection to telecommunication_db in postgresql
connection_string = f"postgres:{pg_password}@localhost:5432/telecommunication_db"
engine = create_engine(f'postgresql://{connection_string}')